**Architecture **

<img src="http://drive.google.com/uc?export=view&id=12JomC2IswVbNGdE0IIvPpUk8vPjP-MBQ"  alt="artchtecture">


---



(1) Importing dependency

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)

Using TensorFlow backend.


(2) Get Data

In [0]:
# x, y = oxflower17.load_data(one_hot=True)
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
#                                                     random_state=42)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


# code test setting
input_shape = (96000, 1)

sample_x = np.zeros([100]+list(input_shape))
sample_y = np.zeros([100]+[50])

print(sample_x.shape, sample_y.shape)

(100, 96000, 1) (100, 50)


(3) Create a sequential model

In [0]:
### Define Parametric Softplus

# alpha * log(1 + exp(beta * x))
def ParametricSoftplus(alpha=0.2, beta=5.0):
  return lambda x: alpha * keras.activations.softplus(beta * x)  

In [0]:
model=Sequential()

# 1st Convolutional Layer (conv1)
model.add(Conv1D (kernel_size=121, filters=48, strides=16, padding='same',
                  input_shape=input_shape))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))
model.add(BatchNormalization())

# Pooling (pool1)
model.add(MaxPooling1D(pool_size=9, strides=4, padding='same'))

# 2nd Convolutional Layer (conv2)
model.add(Conv1D (kernel_size=25, filters=128, padding='same'))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))
model.add(BatchNormalization())

# Pooling (pool2)
model.add(MaxPooling1D(pool_size=9, strides=4, padding='same'))

# 3rd Convolutional Layer (conv3)
model.add(Conv1D (kernel_size=9, filters=192, padding='same'))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))

# 4rd Convolutional Layer (conv4)
model.add(Conv1D (kernel_size=9, filters=192, padding='same'))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))

# 5rd Convolutional Layer (conv5)
model.add(Conv1D (kernel_size=9, filters=128, padding='same'))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))

# Pooling (pool5)
model.add(MaxPooling1D(pool_size=9, strides=4, padding='same'))

# 1st Dense Layer (full6)
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))
model.add(Dropout(0.5)) # Drop-out value is not specified in the paper

# 2nd Dense Layer (full7)
model.add(Dense(4096))
model.add(Activation(ParametricSoftplus(alpha=0.2, beta=0.5)))
model.add(Dropout(0.5))

# Output Layer (full8)
model.add(Dense(50))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 6000, 48)          5856      
_________________________________________________________________
activation_1 (Activation)    (None, 6000, 48)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 6000, 48)          192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1500, 48)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1500, 128)         153728    
_________________________________________________________________
activation_2 (Activation)    (None, 1500, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1500, 128)         512       
__________

(4) Compile 

In [0]:
adam_with_params = keras.optimizers.Adam(lr=0.0002, beta_1=0.1, beta_2=0.999, 
                                         epsilon=1e-8)


model.compile(loss='binary_crossentropy', optimizer=adam_with_params,
              metrics=['accuracy'])

(5) Train

In [0]:
model.fit(sample_x, sample_y, batch_size=36, epochs=10, verbose=1, validation_split=0.2, shuffle=True)

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 5s 65ms/step - loss: 0.6069 - acc: 0.7395 - val_loss: 0.4013 - val_acc: 1.0000
Epoch 2/10
80/80 [==============================] - 1s 7ms/step - loss: 0.3568 - acc: 0.9983 - val_loss: 0.1792 - val_acc: 1.0000
Epoch 3/10
80/80 [==============================] - 1s 7ms/step - loss: 0.1440 - acc: 1.0000 - val_loss: 0.0379 - val_acc: 1.0000
Epoch 4/10
80/80 [==============================] - 1s 7ms/step - loss: 0.0314 - acc: 1.0000 - val_loss: 0.0095 - val_acc: 1.0000
Epoch 5/10
80/80 [==============================] - 1s 7ms/step - loss: 0.0093 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 6/10
80/80 [==============================] - 1s 7ms/step - loss: 0.0042 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 7/10
80/80 [==============================] - 1s 7ms/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 8/10
80/80 [===========================

In [0]:
# %time pred = model.predict_classes(np.expand_dims(x_test[0], axis=0))
# print('Predict:', pred)

# [loss, accuracy] = model.evaluate(x_test, y_test)
# print('Loss:', loss, 'Accuracy:', accuracy)